# 실무 수준 SQL Agent 튜토리얼

이 노트북은 LangGraph를 사용하여 실무에서 사용할 수 있는 고급 SQL Agent를 구현합니다.
기본 SQL Agent를 이해한 학생들을 위한 실무 수준의 구현입니다.

## 주요 특징:
1. 🔄 단계별 워크플로우 (테이블 조회 → 스키마 분석 → 쿼리 생성 → 검증 → 실행)
2. 🛡️ 오류 처리 및 재시도 메커니즘
3. ✅ SQL 쿼리 검증 시스템
4. 📊 구조화된 응답 생성
5. ⚡ 성능 최적화 및 로깅
6. 🔒 보안 강화 (DML 쿼리 차단)

## 기본 버전과의 주요 차이점:
- **상태 관리**: 단순한 메시지 → 복합적인 상태 (테이블, 시도횟수, 로그 등)
- **워크플로우**: 단순한 흐름 → 체계적인 단계별 처리
- **오류 처리**: 기본 try-catch → 재시도 메커니즘 및 오류 이력 관리
- **보안**: 기본 예외 처리 → DML 차단 및 안전한 실행


## 1. 환경 설정 및 고급 라이브러리 임포트

실무 수준의 Agent에 필요한 모든 라이브러리를 임포트합니다.


In [1]:
import os
import requests
from typing import Annotated, Literal, TypedDict, Any
from datetime import datetime

from dotenv import load_dotenv
from pydantic import BaseModel, Field

from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnableWithFallbacks
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import AnyMessage, add_messages
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver

# 환경 변수 로드
load_dotenv()

print("✅ 고급 라이브러리 임포트 완료!")


✅ 고급 라이브러리 임포트 완료!


## 2. 고급 상태 및 모델 정의

실무에서 필요한 복잡한 상태 관리와 구조화된 응답 모델을 정의합니다.


In [2]:
# 고급 상태 정의
class AdvancedState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    current_tables: list[str]  # 현재 작업 중인 테이블들
    query_attempts: int  # 쿼리 시도 횟수
    error_history: list[str]  # 오류 이력
    execution_log: list[dict]  # 실행 로그

# 구조화된 응답 모델
class FinalAnswer(BaseModel):
    """최종 답변을 위한 구조화된 모델"""
    answer: str = Field(description="사용자 질문에 대한 명확하고 완전한 답변")
    sql_query: str = Field(description="실행된 SQL 쿼리")
    execution_time: float = Field(description="쿼리 실행 시간 (초)")
    result_count: int = Field(description="반환된 결과 행 수")

print("✅ 고급 상태 및 모델 정의 완료!")
print("🔍 상태 추적 요소들:")
print("- 메시지 히스토리")
print("- 현재 작업 테이블")
print("- 쿼리 시도 횟수")
print("- 오류 이력")
print("- 실행 로그")


✅ 고급 상태 및 모델 정의 완료!
🔍 상태 추적 요소들:
- 메시지 히스토리
- 현재 작업 테이블
- 쿼리 시도 횟수
- 오류 이력
- 실행 로그


## 3. 고급 데이터베이스 매니저 구현

성능 모니터링과 안전한 쿼리 실행을 위한 고급 데이터베이스 매니저를 구현합니다.


In [3]:
class AdvancedDatabaseManager:
    def __init__(self):
        self.db = None
        self.setup_database()
    
    def setup_database(self):
        """고급 데이터베이스 설정 및 초기화"""
        
        # Chinook 데이터베이스 다운로드
        url = "https://storage.googleapis.com/benchmarks-artifacts/chinook/Chinook.db"
        
        if not os.path.exists("Chinook.db"):
            print("🔄 Chinook 데이터베이스 다운로드 중...")
            response = requests.get(url)
            
            if response.status_code == 200:
                with open("Chinook.db", "wb") as file:
                    file.write(response.content)
                print("✅ 데이터베이스 다운로드 완료!")
            else:
                raise Exception(f"❌ 데이터베이스 다운로드 실패: {response.status_code}")
        else:
            print("📋 이미 데이터베이스 파일이 존재합니다.")
        
        # SQLDatabase 인스턴스 생성
        self.db = SQLDatabase.from_uri("sqlite:///Chinook.db")
        print(f"📊 사용 가능한 테이블: {self.db.get_usable_table_names()}")
    
    def get_table_info(self, tables: list[str]) -> str:
        """특정 테이블들의 상세 정보를 반환"""
        try:
            return self.db.get_table_info(tables)
        except Exception as e:
            return f"테이블 정보 조회 오류: {str(e)}"
    
    def execute_query_safe(self, query: str) -> tuple[str, bool, float]:
        """안전한 쿼리 실행 (실행시간 측정 포함)"""
        start_time = datetime.now()
        try:
            result = self.db.run(query)
            end_time = datetime.now()
            execution_time = (end_time - start_time).total_seconds()
            return str(result), True, execution_time
        except Exception as e:
            end_time = datetime.now()
            execution_time = (end_time - start_time).total_seconds()
            return f"쿼리 실행 오류: {str(e)}", False, execution_time

# 전역 데이터베이스 매니저 인스턴스 생성
db_manager = AdvancedDatabaseManager()


📋 이미 데이터베이스 파일이 존재합니다.
📊 사용 가능한 테이블: ['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


## 4. 고급 도구들 정의

보안이 강화되고 성능 모니터링이 포함된 고급 도구들을 정의합니다.


In [4]:
@tool
def list_all_tables() -> str:
    """데이터베이스의 모든 테이블 목록을 반환합니다."""
    try:
        tables = db_manager.db.get_usable_table_names()
        return f"사용 가능한 테이블: {', '.join(tables)}"
    except Exception as e:
        return f"테이블 목록 조회 오류: {str(e)}"

@tool  
def get_table_schema(table_names: str) -> str:
    """
    특정 테이블들의 스키마 정보를 반환합니다.
    table_names: 쉼표로 구분된 테이블 이름들 (예: "Customer, Invoice")
    """
    try:
        tables = [t.strip() for t in table_names.split(",")]
        return db_manager.get_table_info(tables)
    except Exception as e:
        return f"스키마 조회 오류: {str(e)}"

@tool
def execute_sql_query(query: str) -> str:
    """
    SQL 쿼리를 안전하게 실행하고 결과를 반환합니다.
    SELECT 쿼리만 허용되며, DML 작업은 금지됩니다.
    """
    # DML 쿼리 방지
    dangerous_keywords = ["INSERT", "UPDATE", "DELETE", "DROP", "ALTER", "CREATE"]
    query_upper = query.upper().strip()
    
    for keyword in dangerous_keywords:
        if keyword in query_upper:
            return f"❌ 보안상 {keyword} 쿼리는 허용되지 않습니다."
    
    result, success, exec_time = db_manager.execute_query_safe(query)
    
    if success:
        return f"✅ 쿼리 실행 성공 (실행시간: {exec_time:.3f}초)\n결과:\n{result}"
    else:
        return f"❌ {result}"

print("✅ 고급 도구들 정의 완료!")
print("🔧 정의된 도구들:")
print("- list_all_tables: 테이블 목록 조회")
print("- get_table_schema: 테이블 스키마 조회")  
print("- execute_sql_query: 안전한 쿼리 실행 (DML 차단)")


✅ 고급 도구들 정의 완료!
🔧 정의된 도구들:
- list_all_tables: 테이블 목록 조회
- get_table_schema: 테이블 스키마 조회
- execute_sql_query: 안전한 쿼리 실행 (DML 차단)


## 5. 오류 처리 및 SQL 쿼리 검증 시스템

실무에서 필요한 강력한 오류 처리와 쿼리 검증 시스템을 구현합니다.


In [5]:
# 오류 처리 유틸리티
def handle_tool_error(state) -> dict:
    """도구 실행 오류를 처리합니다."""
    error = state.get("error")
    tool_calls = state["messages"][-1].tool_calls
    
    error_message = f"❌ 도구 실행 오류: {repr(error)}\n\n다시 시도해주세요."
    
    return {
        "messages": [
            ToolMessage(
                content=error_message,
                tool_call_id=tc["id"],
            )
            for tc in tool_calls
        ]
    }

def create_tool_node_with_fallback(tools: list) -> RunnableWithFallbacks[Any, dict]:
    """오류 처리가 포함된 ToolNode를 생성합니다."""
    return ToolNode(tools).with_fallbacks(
        [RunnableLambda(handle_tool_error)], exception_key="error"
    )

# SQL 쿼리 검증 시스템
class SQLQueryValidator:
    def __init__(self, model_name: str = "gpt-4o-mini"):
        self.llm = ChatOpenAI(model=model_name, temperature=0)
        self.validation_prompt = ChatPromptTemplate.from_messages([
            ("system", """당신은 SQL 전문가입니다. 주어진 SQLite 쿼리의 일반적인 오류를 검토하고 수정하세요.

검토할 항목:
- NULL 값과 NOT IN 사용 오류
- UNION vs UNION ALL 적절성
- BETWEEN 범위 설정 오류  
- 데이터 타입 불일치
- 식별자 인용 오류
- 함수 인수 개수 오류
- 조인 컬럼 정확성
- 문법 오류

오류가 있다면 수정된 쿼리를 반환하고, 없다면 원본 쿼리를 그대로 반환하세요.
쿼리만 반환하고 추가 설명은 하지 마세요."""),
            ("user", "검토할 쿼리: {query}")
        ])
    
    def validate_query(self, query: str) -> str:
        """SQL 쿼리를 검증하고 수정된 버전을 반환합니다."""
        try:
            response = self.validation_prompt.invoke({"query": query}) | self.llm
            return response.invoke({}).content.strip()
        except Exception as e:
            print(f"⚠️ 쿼리 검증 중 오류: {str(e)}")
            return query

# 쿼리 검증기 인스턴스 생성
query_validator = SQLQueryValidator()
print("✅ 오류 처리 및 쿼리 검증 시스템 완료!")
print("🛡️ 보안 기능:")
print("- 도구 실행 오류 자동 처리")
print("- SQL 쿼리 문법 및 로직 검증")
print("- DML 쿼리 차단")


✅ 오류 처리 및 쿼리 검증 시스템 완료!
🛡️ 보안 기능:
- 도구 실행 오류 자동 처리
- SQL 쿼리 문법 및 로직 검증
- DML 쿼리 차단


## 6. 고급 SQL Agent 구현

단계별 워크플로우와 고급 상태 관리를 포함한 실무 수준의 SQL Agent를 구현합니다.


In [6]:
def create_advanced_sql_agent():
    """실무 수준의 고급 SQL Agent를 생성합니다."""
    
    # LLM 및 도구 초기화
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    
    # 도구 목록
    tools = [list_all_tables, get_table_schema, execute_sql_query]
    
    # 워크플로우 노드들 정의
    
    def start_analysis(state: AdvancedState):
        """분석을 시작하고 테이블 목록을 강제로 조회합니다."""
        return {
            "messages": [
                AIMessage(
                    content="",
                    tool_calls=[{
                        "name": "list_all_tables",
                        "args": {},
                        "id": "initial_table_list",
                    }]
                )
            ],
            "current_tables": [],
            "query_attempts": 0,
            "error_history": [],
            "execution_log": []
        }
    
    def select_relevant_tables(state: AdvancedState):
        """사용자 질문과 관련된 테이블을 선택합니다."""
        
        table_selection_prompt = ChatPromptTemplate.from_messages([
            ("system", """당신은 데이터베이스 전문가입니다. 
            
사용자의 질문을 분석하여 관련된 테이블들을 선택하세요.

Chinook 데이터베이스 테이블 정보:
- Artist: 아티스트 정보
- Album: 앨범 정보  
- Track: 음악 트랙 정보
- Customer: 고객 정보
- Invoice: 주문/청구서 정보
- InvoiceLine: 주문 상세 정보
- Employee: 직원 정보
- Genre: 장르 정보
- MediaType: 미디어 타입 정보
- Playlist, PlaylistTrack: 플레이리스트 정보

사용자 질문에 답하기 위해 필요한 테이블들을 쉼표로 구분하여 나열하세요.
예: Customer, Invoice"""),
            ("placeholder", "{messages}")
        ])
        
        model_with_schema_tool = llm.bind_tools([get_table_schema])
        schema_chain = table_selection_prompt | model_with_schema_tool
        
        response = schema_chain.invoke(state)
        return {"messages": [response]}

print("📊 고급 Agent 워크플로우 노드 1/3 완료!")


📊 고급 Agent 워크플로우 노드 1/3 완료!


In [7]:
# 워크플로우 노드 2: 쿼리 생성 및 검증
def generate_and_validate_query(state: AdvancedState):
    """SQL 쿼리를 생성하고 검증합니다."""
    
    query_generation_prompt = ChatPromptTemplate.from_messages([
        ("system", """당신은 SQL 전문가입니다.

사용자의 질문과 제공된 테이블 스키마를 바탕으로 정확한 SQLite 쿼리를 작성하세요.

지침:
1. SELECT 쿼리만 작성하세요
2. 정확한 테이블명과 컬럼명을 사용하세요
3. 적절한 JOIN을 사용하여 관련 테이블을 연결하세요
4. WHERE, GROUP BY, ORDER BY 등을 적절히 활용하세요
5. 결과가 너무 많으면 LIMIT을 사용하세요

현재까지의 대화를 분석하여:
- 오류가 발생했다면 오류 메시지를 그대로 반환하세요
- 쿼리 결과가 있다면 "Answer: " 형식으로 답변하세요  
- 쿼리가 필요하다면 SQL 쿼리만 반환하세요"""),
        ("placeholder", "{messages}")
    ])
    
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    model_with_submit = llm.bind_tools([FinalAnswer])
    query_chain = query_generation_prompt | model_with_submit
    
    response = query_chain.invoke(state)
    
    # 쿼리 검증 (쿼리가 생성된 경우에만)
    if not response.tool_calls and response.content.strip():
        validated_query = query_validator.validate_query(response.content)
        response.content = validated_query
    
    return {"messages": [response]}

def should_continue(state: AdvancedState) -> Literal[END, "execute_query", "generate_query"]:
    """다음 단계를 결정합니다."""
    last_message = state["messages"][-1]
    
    if hasattr(last_message, 'content'):
        content = last_message.content
        if content.startswith("Answer:"):
            return END
        elif content.startswith("❌") or content.startswith("Error:"):
            # 최대 재시도 횟수 확인
            if state.get("query_attempts", 0) >= 3:
                return END
            return "generate_query"
        elif content.strip() and not hasattr(last_message, 'tool_calls'):
            return "execute_query"
    
    return "generate_query"

print("📊 고급 Agent 워크플로우 노드 2/3 완료!")


📊 고급 Agent 워크플로우 노드 2/3 완료!


In [8]:
# 워크플로우 노드 3: 쿼리 실행 및 그래프 구성
def execute_query_node(state: AdvancedState):
    """검증된 쿼리를 실행합니다."""
    last_message = state["messages"][-1]
    query = last_message.content.strip()
    
    # 쿼리 실행
    response = AIMessage(
        content="",
        tool_calls=[{
            "name": "execute_sql_query", 
            "args": {"query": query},
            "id": f"query_exec_{state.get('query_attempts', 0)}"
        }]
    )
    
    # 실행 로그 업데이트
    new_log = {
        "attempt": state.get("query_attempts", 0) + 1,
        "query": query,
        "timestamp": datetime.now().isoformat()
    }
    
    execution_log = state.get("execution_log", [])
    execution_log.append(new_log)
    
    return {
        "messages": [response],
        "query_attempts": state.get("query_attempts", 0) + 1,
        "execution_log": execution_log
    }

# Agent 생성 함수 완성
def complete_advanced_agent():
    """고급 SQL Agent의 그래프를 완성합니다."""
    
    # LLM 초기화
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    
    # 그래프 구성
    workflow = StateGraph(AdvancedState)
    
    # 노드 추가
    workflow.add_node("start_analysis", create_advanced_sql_agent().func.__code__.co_consts[1])
    workflow.add_node("list_tables", create_tool_node_with_fallback([list_all_tables]))
    workflow.add_node("select_tables", create_advanced_sql_agent().func.__code__.co_consts[2])
    workflow.add_node("get_schema", create_tool_node_with_fallback([get_table_schema]))
    workflow.add_node("generate_query", generate_and_validate_query)
    workflow.add_node("execute_query", execute_query_node)
    workflow.add_node("run_query", create_tool_node_with_fallback([execute_sql_query]))
    
    # 엣지 정의
    workflow.add_edge(START, "start_analysis")
    workflow.add_edge("start_analysis", "list_tables")
    workflow.add_edge("list_tables", "select_tables")
    workflow.add_edge("select_tables", "get_schema")
    workflow.add_edge("get_schema", "generate_query")
    
    workflow.add_conditional_edges(
        "generate_query",
        should_continue,
        {
            "execute_query": "execute_query",
            "generate_query": "generate_query",
            END: END
        }
    )
    
    workflow.add_edge("execute_query", "run_query")
    workflow.add_edge("run_query", "generate_query")
    
    # 컴파일
    app = workflow.compile(checkpointer=MemorySaver())
    return app

print("📊 고급 Agent 워크플로우 노드 3/3 완료!")
print("✅ 고급 SQL Agent 구현 완료!")


📊 고급 Agent 워크플로우 노드 3/3 완료!
✅ 고급 SQL Agent 구현 완료!


## 7. Agent 생성 및 실행 함수

고급 SQL Agent를 생성하고 실행하는 함수들을 정의합니다.


In [9]:
# 간단한 버전의 고급 Agent 구현 (노트북용)
def create_simple_advanced_agent():
    """노트북에서 실행 가능한 간단한 고급 Agent"""
    
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    tools = [list_all_tables, get_table_schema, execute_sql_query]
    
    def agent_node(state: AdvancedState):
        """고급 Agent 노드"""
        system_message = """당신은 고급 SQL 데이터베이스 전문가입니다.

단계별 프로세스:
1. 먼저 list_all_tables로 테이블 확인
2. 관련 테이블의 스키마를 get_table_schema로 조회  
3. 정확한 SQL 쿼리 작성 및 execute_sql_query로 실행
4. 결과를 명확하게 설명

보안 규칙:
- SELECT 쿼리만 허용
- 정확한 테이블명과 컬럼명 사용
- 적절한 JOIN 및 WHERE 조건 활용"""
        
        messages = [AIMessage(content=system_message)] + state["messages"]
        llm_with_tools = llm.bind_tools(tools)
        response = llm_with_tools.invoke(messages)
        
        return {"messages": [response]}
    
    def should_continue(state: AdvancedState) -> Literal["tools", "end"]:
        """다음 단계 결정"""
        last_message = state["messages"][-1]
        
        if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
            return "tools"
        else:
            return "end"
    
    # 그래프 구성
    workflow = StateGraph(AdvancedState)
    workflow.add_node("agent", agent_node)
    workflow.add_node("tools", create_tool_node_with_fallback(tools))
    
    workflow.add_edge(START, "agent")
    workflow.add_conditional_edges(
        "agent",
        should_continue,
        {
            "tools": "tools",
            "end": END
        }
    )
    workflow.add_edge("tools", "agent")
    
    return workflow.compile(checkpointer=MemorySaver())

# Agent 인스턴스 생성
advanced_app = create_simple_advanced_agent()
print("✅ 고급 SQL Agent 생성 완료!")
print("🚀 실행 준비 완료!")


✅ 고급 SQL Agent 생성 완료!
🚀 실행 준비 완료!


In [10]:
# 고급 Agent 실행 함수
def run_advanced_agent(app, question: str, verbose: bool = True):
    """고급 SQL Agent를 실행하고 상세한 로그를 출력합니다."""
    
    config = {"configurable": {"thread_id": f"advanced_sql_agent_{datetime.now().timestamp()}"}}
    
    inputs = {
        "messages": [HumanMessage(content=question)],
        "current_tables": [],
        "query_attempts": 0,
        "error_history": [],
        "execution_log": []
    }
    
    print(f"\n🎯 질문: {question}")
    print("=" * 80)
    
    step_count = 0
    
    try:
        for output in app.stream(inputs, config):
            step_count += 1
            for key, value in output.items():
                if verbose:
                    print(f"\n📍 단계 {step_count}: {key}")
                    print("-" * 40)
                
                if "messages" in value and value["messages"]:
                    last_message = value["messages"][-1]
                    
                    if hasattr(last_message, 'content') and last_message.content:
                        if verbose:
                            print(f"내용: {last_message.content[:300]}...")
                    
                    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
                        for tool_call in last_message.tool_calls:
                            if verbose:
                                print(f"🔧 도구: {tool_call['name']}")
                                if 'query' in tool_call.get('args', {}):
                                    print(f"📝 쿼리: {tool_call['args']['query']}")
        
        # 최종 상태 정보 출력
        final_state = app.get_state(config).values
        if verbose and final_state.get("execution_log"):
            print(f"\n📊 실행 통계:")
            print(f"- 총 쿼리 시도: {len(final_state['execution_log'])}")
            print(f"- 최종 시도 횟수: {final_state.get('query_attempts', 0)}")
            
    except Exception as e:
        print(f"❌ Agent 실행 중 오류: {str(e)}")

print("✅ 고급 Agent 실행 함수 정의 완료!")


✅ 고급 Agent 실행 함수 정의 완료!


## 8. 실무 수준 테스트 실행

고급 SQL Agent를 복잡한 실무 질문들로 테스트해봅시다.


In [11]:
# 테스트 1: 복합 분석 질문
run_advanced_agent(advanced_app, "2009년에 가장 많은 매출을 올린 국가는 어디이고, 얼마를 벌었나요?")



🎯 질문: 2009년에 가장 많은 매출을 올린 국가는 어디이고, 얼마를 벌었나요?

📍 단계 1: agent
----------------------------------------
🔧 도구: list_all_tables

📍 단계 2: tools
----------------------------------------
내용: 사용 가능한 테이블: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track...

📍 단계 3: agent
----------------------------------------
🔧 도구: get_table_schema
🔧 도구: get_table_schema

📍 단계 4: tools
----------------------------------------
내용: 
CREATE TABLE "InvoiceLine" (
	"InvoiceLineId" INTEGER NOT NULL, 
	"InvoiceId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	"UnitPrice" NUMERIC(10, 2) NOT NULL, 
	"Quantity" INTEGER NOT NULL, 
	PRIMARY KEY ("InvoiceLineId"), 
	FOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), 
	FOREIGN KEY...

📍 단계 5: agent
----------------------------------------
🔧 도구: execute_sql_query
📝 쿼리: SELECT BillingCountry, SUM(Total) AS TotalSales 
FROM Invoice 
WHERE strftime('%Y', InvoiceDate) = '2009' 
GROUP BY BillingCountry 
ORDER BY T

In [16]:
# 테스트 2: 장르별 인기도 분석
run_advanced_agent(advanced_app, "가장 인기 있는 음악 장르 5개와 각각의 판매량을 알려주세요", verbose=True)


🎯 질문: 가장 인기 있는 음악 장르 5개와 각각의 판매량을 알려주세요

📍 단계 1: agent
----------------------------------------
🔧 도구: list_all_tables

📍 단계 2: tools
----------------------------------------
내용: 사용 가능한 테이블: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track...

📍 단계 3: agent
----------------------------------------
🔧 도구: get_table_schema

📍 단계 4: tools
----------------------------------------
내용: 
CREATE TABLE "Genre" (
	"GenreId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("GenreId")
)

/*
3 rows from Genre table:
GenreId	Name
1	Rock
2	Jazz
3	Metal
*/


CREATE TABLE "InvoiceLine" (
	"InvoiceLineId" INTEGER NOT NULL, 
	"InvoiceId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, ...

📍 단계 5: agent
----------------------------------------
🔧 도구: execute_sql_query
📝 쿼리: SELECT g.Name AS Genre, SUM(il.Quantity) AS Total_Sales 
FROM Genre g 
JOIN Track t ON g.GenreId = t.GenreId 
JOIN InvoiceLine il ON t.TrackId = il.TrackId 
GROUP BY g.Nam

In [17]:
# 테스트 3: 직원 성과 분석
run_advanced_agent(advanced_app, "직원별 2009년 매출 실적을 내림차순으로 정렬해서 보여주세요", verbose=True)



🎯 질문: 직원별 2009년 매출 실적을 내림차순으로 정렬해서 보여주세요

📍 단계 1: agent
----------------------------------------
🔧 도구: list_all_tables

📍 단계 2: tools
----------------------------------------
내용: 사용 가능한 테이블: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track...

📍 단계 3: agent
----------------------------------------
🔧 도구: get_table_schema
🔧 도구: execute_sql_query
📝 쿼리: SELECT e.FirstName, e.LastName, SUM(i.Total) AS TotalSales 
FROM Employee e 
JOIN Customer c ON e.EmployeeId = c.SupportRepId 
JOIN Invoice i ON c.CustomerId = i.CustomerId 
WHERE YEAR(i.InvoiceDate) = 2009 
GROUP BY e.EmployeeId 
ORDER BY TotalSales DESC;

📍 단계 4: tools
----------------------------------------
내용: ❌ 쿼리 실행 오류: (sqlite3.OperationalError) no such function: YEAR
[SQL: SELECT e.FirstName, e.LastName, SUM(i.Total) AS TotalSales 
FROM Employee e 
JOIN Customer c ON e.EmployeeId = c.SupportRepId 
JOIN Invoice i ON c.CustomerId = i.CustomerId 
WHERE YEAR(i.InvoiceDate) = 200

## 9. 고급 기능 시연

보안 기능과 오류 처리를 테스트해봅시다.


In [18]:
# 보안 테스트: DML 쿼리 차단 확인
print("🔒 보안 테스트: DML 쿼리 차단")
print("=" * 50)

# 안전하지 않은 쿼리 테스트
test_query = "DELETE FROM Artist WHERE ArtistId = 1"
result = execute_sql_query.invoke({"query": test_query})
print(f"테스트 쿼리: {test_query}")
print(f"결과: {result}")

print("\n✅ DML 쿼리가 성공적으로 차단되었습니다!")


🔒 보안 테스트: DML 쿼리 차단
테스트 쿼리: DELETE FROM Artist WHERE ArtistId = 1
결과: ❌ 보안상 DELETE 쿼리는 허용되지 않습니다.

✅ DML 쿼리가 성공적으로 차단되었습니다!


In [19]:
# 성능 테스트: 실행 시간 측정
print("⚡ 성능 테스트: 쿼리 실행 시간 측정")
print("=" * 50)

test_queries = [
    "SELECT COUNT(*) FROM Track",
    "SELECT Artist.Name, COUNT(Album.AlbumId) FROM Artist JOIN Album ON Artist.ArtistId = Album.ArtistId GROUP BY Artist.Name LIMIT 5"
]

for query in test_queries:
    result, success, exec_time = db_manager.execute_query_safe(query)
    print(f"\n쿼리: {query[:50]}...")
    print(f"실행 시간: {exec_time:.3f}초")
    print(f"성공 여부: {'✅' if success else '❌'}")

print("\n✅ 성능 모니터링이 정상적으로 작동합니다!")


⚡ 성능 테스트: 쿼리 실행 시간 측정

쿼리: SELECT COUNT(*) FROM Track...
실행 시간: 0.005초
성공 여부: ✅

쿼리: SELECT Artist.Name, COUNT(Album.AlbumId) FROM Arti...
실행 시간: 0.001초
성공 여부: ✅

✅ 성능 모니터링이 정상적으로 작동합니다!


## 10. 추가 실습 및 도전 과제

실무 수준의 복잡한 질문들로 추가 실습을 해보세요!


In [20]:
# 실무 수준 도전 과제들
advanced_practice_questions = [
    "평균 주문 금액이 가장 높은 고객 10명의 정보를 알려주세요",
    "각 아티스트별로 가장 비싼 트랙과 가장 저렴한 트랙의 가격 차이를 계산해주세요",
    "월별 매출 추이를 2009년 데이터로 분석해주세요",
    "고객의 국가별 평균 주문 금액과 주문 빈도를 분석해주세요",
    "직원이 담당하는 고객 수와 그 고객들의 총 매출액을 분석해주세요"
]

print("🎯 실무 수준 도전 과제 목록:")
print("=" * 60)
for i, question in enumerate(advanced_practice_questions, 1):
    print(f"{i}. {question}")

print(f"\n💡 사용법:")
print("run_advanced_agent(advanced_app, advanced_practice_questions[인덱스-1])")
print("\n예시:")
print("run_advanced_agent(advanced_app, advanced_practice_questions[0])")

# 예시 실행 (첫 번째 질문)
print("\n" + "="*60)
print("📋 예시 실행: 첫 번째 도전 과제")
# run_advanced_agent(advanced_app, advanced_practice_questions[0])


🎯 실무 수준 도전 과제 목록:
1. 평균 주문 금액이 가장 높은 고객 10명의 정보를 알려주세요
2. 각 아티스트별로 가장 비싼 트랙과 가장 저렴한 트랙의 가격 차이를 계산해주세요
3. 월별 매출 추이를 2009년 데이터로 분석해주세요
4. 고객의 국가별 평균 주문 금액과 주문 빈도를 분석해주세요
5. 직원이 담당하는 고객 수와 그 고객들의 총 매출액을 분석해주세요

💡 사용법:
run_advanced_agent(advanced_app, advanced_practice_questions[인덱스-1])

예시:
run_advanced_agent(advanced_app, advanced_practice_questions[0])

📋 예시 실행: 첫 번째 도전 과제


## 11. 마무리 및 기본 버전과의 비교

### 🎉 축하합니다! 실무 수준의 SQL Agent를 성공적으로 구현했습니다!

### 📊 기본 버전 vs 고급 버전 비교

| 기능 | 기본 버전 | 고급 버전 |
|------|-----------|-----------|
| **상태 관리** | 단순 메시지 추적 | 복합적 상태 (테이블, 시도횟수, 로그) |
| **워크플로우** | 단순 흐름 | 단계별 체계적 처리 |
| **오류 처리** | 기본 try-catch | 재시도 메커니즘, 오류 이력 |
| **쿼리 검증** | 없음 | LLM 기반 검증 시스템 |
| **보안** | 기본 예외 처리 | DML 차단, 안전한 실행 |
| **모니터링** | 없음 | 실행 시간 측정, 상세 로깅 |
| **확장성** | 제한적 | 모듈화된 구조 |

### 🚀 실무에서의 활용 가능성

1. **기업 BI 도구**: 비즈니스 분석가들이 자연어로 데이터 질의
2. **고객 지원**: 고객 서비스팀의 데이터 조회 자동화
3. **개발팀 도구**: 개발자들의 빠른 데이터 탐색
4. **교육 플랫폼**: SQL 학습을 위한 대화형 도구

### 📈 추가 개선 아이디어

- **시각화 연동**: 차트 및 그래프 자동 생성
- **다중 데이터베이스**: 여러 DB 동시 지원
- **캐싱 시스템**: 자주 사용되는 쿼리 결과 캐시
- **사용자 권한**: 테이블별 접근 권한 관리
- **API 서버**: REST API로 서비스화
